In [242]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import time
import json
import requests
import os

In [243]:
# Get data.
df = pd.read_csv('rte_rawdata.csv')
plants_df = pd.read_csv('plants.csv')

In [244]:
# Remove columns.
df = df[['EventID', 'EventStart', 'Unit Name']]
df.head()

,EventID,EventStart,Unit Name
0,04424_20101226T163810,12/26/2010 16:30,ST LAURENT 2
1,04424_20101231T184352,12/31/2010 18:30,BUGEY 2
2,04424_20101231T194146,12/31/2010 19:30,CRUAS 2
3,04424_20110102T120033,12/31/2010 19:30,CRUAS 2
4,04424_20101231T184352,12/31/2010 18:30,BUGEY 2


In [245]:
# Join events with plants data.
df = df.merge(plants_df)
df.head()

,EventID,EventStart,Unit Name,Latitude,Longitude
0,04424_20101226T163810,12/26/2010 16:30,ST LAURENT 2,47.72,1.5775
1,04424_13909,2/22/2011 1:10,ST LAURENT 2,47.72,1.5775
2,04424_15208,6/28/2011 14:20,ST LAURENT 2,47.72,1.5775
3,04424_16190,10/9/2011 4:10,ST LAURENT 2,47.72,1.5775
4,04424_16209,10/10/2011 23:00,ST LAURENT 2,47.72,1.5775


In [246]:
df.drop_duplicates('EventID', inplace=True)
len(df)

3887

In [247]:
output_folder = 'weather_output'

def get_file_path(id):
    return output_folder + '/' + id + '.json'

for row in df.values:
    # Create folder if it doesn't exist.
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
        
    # Check if file exists, if so, continue.
    file_path = get_file_path(row[0])
    if os.path.exists(file_path):
        continue
        
    print('Creating', file_path)
    
    # Convert string to date time.
    dt = datetime.strptime(row[1], "%m/%d/%Y %H:%M")
    
    # Convert date time to UNIX timestamp.
    t = time.mktime(dt.timetuple())

    # Create URL.
    url = 'https://api.darksky.net/forecast/0123456789abcdef9876543210fedcba/' + str(row[3]) + ',' + str(row[4]) + ',' + str(int(t)) + '?exclude=currently,minutely,hourly,alerts,flags'
    
    # Get and parse response.
    response = requests.get(url)   
    json_data = json.loads(response.text)
  
    # Write file.
    with open(file_path, 'w') as fp:
        json.dump(json_data['daily']['data'], fp)
    
    

In [248]:
result = []

for row in df.values:
    # Get file.
    file_path = get_file_path(row[0])
    json_data = json.load(open(file_path))
    
    temperature_low = None
    temperature_high = None
    cloud_cover = None
    pressure = None
    precip_prob = None
    precip_intensity = None
    precip_type = 'None'
    visibility = None
    moon_phase = None
    
    
    if ('temperatureLow' in json_data[0]):
        temperature_low = json_data[0]['temperatureLow']
    if ('temperatureHigh' in json_data[0]):
        temperature_high = json_data[0]['temperatureHigh']
    if ('cloudCover' in json_data[0]):
        cloud_cover = json_data[0]['cloudCover']
    if ('pressure' in json_data[0]):
        pressure = json_data[0]['pressure']
    if ('precipProbability' in json_data[0]):
        precip_prob = json_data[0]['precipProbability']
    if ('precipIntensity' in json_data[0]):
        precip_intensity = json_data[0]['precipIntensity']
    if ('precipType' in json_data[0]):
        precip_type = json_data[0]['precipType']
    if ('visibility' in json_data[0]):
        visibility = json_data[0]['visibility']
    if ('moonPhase' in json_data[0]):
        moon_phase = json_data[0]['moonPhase']
    
#     print(row[0])
    
    # Append data.
    result.append([
        temperature_low,
        temperature_high,
        cloud_cover,
        pressure,
        json_data[0]['humidity'],
        json_data[0]['dewPoint'],
        json_data[0]['windBearing'],
        json_data[0]['windSpeed'],
        precip_prob,
        precip_intensity,
        precip_type,
        visibility,
        moon_phase
    ])
    
# Convert list to np array.
result = np.array(result)

In [249]:
# Assign result to series.
df['TempLow'] = result[:,0]
df['TempHigh'] = result[:,1]
df['CloudCover'] = result[:,2]
df['Pressure'] = result[:,3]
df['Humidity'] = result[:,4]
df['DewPoint'] = result[:,5]
df['WindBearing'] = result[:,6]
df['WindSpeed'] = result[:,7]
df['PrecipProb'] = result[:,8]
df['PrecipIntensity'] = result[:,9]
df['PrecipType'] = result[:,10]
df['Visibility'] = result[:,11]
df['MoonPhase'] = result[:,12]

df.head()

,EventID,EventStart,Unit Name,Latitude,Longitude,TempLow,TempHigh,CloudCover,Pressure,Humidity,DewPoint,WindBearing,WindSpeed,PrecipProb,PrecipIntensity,PrecipType,Visibility,MoonPhase
0,04424_20101226T163810,12/26/2010 16:30,ST LAURENT 2,47.72,1.5775,26.66,34.28,0.43,1029.75,0.89,23.51,326,0.5,0,0,None,5.73,0.68
1,04424_13909,2/22/2011 1:10,ST LAURENT 2,47.72,1.5775,32.6,44.81,0.81,1015.71,0.85,37.39,18,4.53,0.25,0.0004,rain,4.74,0.65
2,04424_15208,6/28/2011 14:20,ST LAURENT 2,47.72,1.5775,56.47,90.1,0.37,1013.77,0.62,63.29,266,4.67,0.38,0.0028,rain,7.83,0.9
3,04424_16190,10/9/2011 4:10,ST LAURENT 2,47.72,1.5775,57.22,63.24,0.95,1023.98,0.88,54.86,281,9.35,0.33,0.0013,rain,6.97,0.42
4,04424_16209,10/10/2011 23:00,ST LAURENT 2,47.72,1.5775,58.21,61.85,0.97,1026.12,0.85,54.84,265,9.73,0,0,None,7.72,0.45


In [250]:
valid = len(df.dropna(subset=['CloudCover', 'TempLow', 'TempHigh', 'PrecipProb', 'PrecipIntensity', 
                              'Pressure', 'PrecipType', 'Visibility', 'MoonPhase']))
total = len(df)
print('Valid:', valid, 'of', total)
print(valid / total * 100, '%')

Valid: 3781 of 3887
97.27296115255982 %


In [251]:
# Drop columns.
df.drop(['Unit Name', 'EventStart', 'Latitude', 'Longitude'], axis=1, inplace=True)

In [252]:
df.to_csv('weather.csv', index=False)